In [1]:
from ipyleaflet import Map, basemaps, basemap_to_tiles

m = Map(
    basemap=basemap_to_tiles(basemaps.NASAGIBS.ModisTerraTrueColorCR, "2017-04-08"),
    center=(52.204793, 360.121558),
    zoom=4
)

Map(center=[52.204793, 360.121558], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title'…

In [3]:
import pandas as pd
import ipywidgets as widgets
from ipywidgets.widgets import Layout, HBox, VBox
import bqplot as bqp 
import bqplot.pyplot as plt

In [4]:
data_df = pd.read_csv('data.csv')

In [5]:
# data_df = pd.read_excel("data.xlsx", sheet_name="Sheet1")
# data_df = data_df[~data_df["YearFrom"].isin(["16..","17.."])]  
# data_df = data_df.astype({"YearFrom": float})
# data_df['VerdachteLeeftijd/AccusedAge'] = data_df['VerdachteLeeftijd/AccusedAge'].str.extract('(\d+)')
# data_df["VerdachteLeeftijd/AccusedAge"] = pd.to_numeric(data_df["VerdachteLeeftijd/AccusedAge"])
# data_df.to_csv('data.csv', index=False)

In [6]:
profession_list = ['Baker',
 'Bricklayer',
 '(wool)weaver',
 'Soldier',
 'Farmer',
 'Shoemaker',
 'Pinmaker',
 'knitter',
 'Tailor',
 'Merchant',
 'Doctor',
 'Innkeeper',
 'Vendor',
 'Servant',
 'Lacemaker',
 'City-alderman',
 'Hunter',
 'Driver',
 'Beer brewer',
 'Dressmaker',
 'Sailor_Captain',
 'Miller',
 'bailiff',
 'Carpandter',
 'Advocaat',
 'Prisonguard',
 'shepherd']

crime_list = [ 'Threat',
 'Ill-treatment',
 'Begging',
 'Killing',
 'Burglary',
 'Robbery',
 'Extortion',
 'Fraud',
 'Theft',
 'Looting',
 'Offending',
 'Prostitution',
 'Outlawry',
 'Violence',
 'Resistance',
 'Adultery',
 'Disturb public order',
 'Assault',
 'Destruction',
 'Illegal',
 'Suicide',
 'Barrage',
 'Trading with enemy',
 'Cheating',
 'Bigamy',
 'Evasion',
 'Escape',
 'Kidnapping',
 'Forbidden',
 'Refusing',
 'Violating']

punishment_list = [ 'fine',
 'acquittal',
 'death penalty',
 'banishment',
 'branding (with a hot iron mark)',
 'on display',
 'costs',
 'decapitation',
 'prison',
 'dismissed',
 'discharged',
 'hanging',
 'floggging',
 'baked',
 'dragged',
 'pillory',
 'out of the city',
 'expiration of office',
 'confinement',
 'remand',
 'confiscation of property',
 'cut off hand',
 'diverse',
 'around the Gewandhuis',
 'ask forgiveness from God and the Justice',
 'city running',
 'conditional release',
 'pilgrimage']

place_list = ['Aachen',
 'Aalst',
 'Aarle',
 'Alem',
 'Almkerk',
 'Alphen',
 'Amsterdam',
 'Anne Cathrien',
 'Antwerp',
 'Arnhem',
 'Bakel',
 'Bavel',
 'Beek-en-Donk',
 'Bergeijk',
 'Berghem',
 'Berlicum',
 'Best',
 'Bestbroek',
 'Bestenbroek',
 'Blaarthem',
 'BladelBlok',
 'Bocholtz',
 'Boekel',
 'Boerdonk',
 'Bokhoven',
 'Bommelerwaard',
 'Borkel',
 'Boxtel',
 'Boxtelseheide',
 'Brabant',
 'Breda',
 'Budel',
 'Castéron',
 'Coesfeld',
 'Coevering',
 'Cox',
 'Craijven',
 'Crevecoeur',
 'Cromvoirt',
 'Cuijk',
 'Den Dungen',
 'Deurne',
 'Deuteren',
 'Diessen',
 'Dinther',
 'Doeveren',
 'Dordrecht',
 'Driel',
 'Drunen',
 'Duizel',
 'Eerde',
 'Eersel',
 'Eibergen',
 'Eikendonk',
 'Eindhoven',
 'Empel',
 'Enschot',
 'Erp',
 'Esch',
 'Etten',
 'Fellenoort',
 'Gennepse',
 'Gestel',
 'Gimonde',
 'Ginneken',
 'Gorinchem',
 'Grave',
 'Groenstraat',
 'Haaren',
 'Haarsteeg',
 'Halder',
 'Halsteren',
 'Hamont-Achel',
 'Handel',
 'Hapert',
 'Haren',
 'Hedel',
 'Heel',
 'Heerenberg',
 'Helmond',
 'Helvoirt',
 'Helvoirt.1',
 'Hendren',
 'herberg',
 'Herlaar',
 'Herpen',
 'Herpt',
 'Heukelom',
 'Heusden',
 'Hilvarenbeek',
 'Hintham',
 'Hokkelen',
 'Holland',
 'Hooge Mierde',
 'Hoogeloon',
 'Hoog-Gemonde',
 'inbraak',
 'Jan Jansen Eersel',
 'Jekschotse',
 'Kempenland',
 'Kerkhoven',
 'Kessel',
 'Kevelaer',
 'Lage-Mierde',
 'Land',
 'Leende',
 'Leiden',
 'Liempde',
 'Lierop',
 'Lieshout',
 'Linter',
 'Lith',
 'Lithoijen',
 'Lommel',
 'Londen',
 'Loon',
 'Luikerland',
 'Luyksgestel',
 'Maasland',
 'Maastricht',
 'Maren',
 'Marene',
 'Markiezaat',
 'Markt',
 'Mechelen',
 'Megen',
 'Meierij',
 'Middelbeers',
 'Mierde',
 'Milheeze',
 'Neerpelt',
 'Netersel',
 'Nieuwkuijk',
 'Nijmegen',
 'Nistelrode',
 'Nistelrooijse',
 'North Brabant',
 'North Holland',
 'Nuenen',
 'Nuland',
 'Oerle',
 'Oijen',
 'Oirschot',
 'Oisterwijk',
 'Olmen',
 'onbekend',
 'Oost',
 'Oostelbeer',
 'Oosterhout',
 'Opole Voivodeship',
 'Opwetten',
 'Orthensedijk',
 'Oss',
 'Peer',
 'Pekel',
 'Poppel',
 'Postel Abbey',
 'Postelle',
 'Ravenswaaijse',
 'Retie',
 'Reusel',
 'Riel',
 'Riethoven',
 'Rosmalen',
 'Rosmalen.1',
 'Helvoirt.2',
 'Rosmalenseveld',
 'Rotterdam',
 'Sambeek',
 'Schaft',
 'Schijndel',
 'Schutlakensedam',
 's-Hertogenbosch',
 'Sint-Andries',
 'Sint-Antonis',
 'Sint-Michielsgestel',
 'Sint-Oedenrode',
 'Someren',
 'Somerense',
 'Son',
 'Spa',
 'Sprang',
 'Stabroek',
 'Steensel',
 'Stratum',
 'The Hague',
 'Tilburg',
 'Tongelre',
 'Turnhoutseheide',
 'Uden',
 'Udenhout',
 'Ulvi',
 'Veere',
 'Veghel',
 'Veldhoven',
 'vele plaatsen',
 'Vierlingsbeek',
 'Vilvoordev',
 'Vlierden',
 'Vlijmen',
 'Vorstenbosch',
 'Vught',
 'Walik',
 'Westerhoven',
 'Wijbosch',
 'Winkel',
 'Wintelre',
 'Woensel',
 'Wolfswinkel',
 'Zaltbommel',
 'Zandoerle',
 'Zandstraat',
 'Zeeland',
 'Zeelst']

In [7]:
year_range=widgets.IntRangeSlider(
    value=[1500, 1803],
    min=1500, max=1803, step=1,
    description='Select Year:',
    layout=Layout(width='960px'),
    continuous_update=False)

years_filter = widgets.Button(
    description='',
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Click to unselect',
    icon='check' # (FontAwesome names without the `fa-` prefix)
)

age_filter = widgets.Button(
    description='',
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Click to unselect',
    icon='check' # (FontAwesome names without the `fa-` prefix)
)

gender_filter = widgets.Button(
    description='',
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Click to unselect',
    icon='check' # (FontAwesome names without the `fa-` prefix)
)

profession_filter = widgets.Button(
    description='',
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Click to unselect',
    icon='check' # (FontAwesome names without the `fa-` prefix)
)

place_filter = widgets.Button(
    description='',
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Click to unselect',
    icon='check' # (FontAwesome names without the `fa-` prefix)
)

crime_filter = widgets.Button(
    description='',
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Click to unselect',
    icon='check' # (FontAwesome names without the `fa-` prefix)
)

punishment_filter = widgets.Button(
    description='',
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Click to unselect',
    icon='check' # (FontAwesome names without the `fa-` prefix)
)

In [8]:
data_df[place_list] = data_df[place_list].replace({0:None})
data_df[punishment_list] = data_df[punishment_list].replace({0:None})

crime_df =data_df[crime_list].apply(pd.Series.value_counts).iloc[0].rename_axis('unique_values').reset_index(name='counts')
crime_df = crime_df.sort_values('counts', ascending=False).head(5)
punishment_df =data_df[punishment_list].apply(pd.Series.value_counts).iloc[0].rename_axis('unique_values').reset_index(name='counts')
punishment_df = punishment_df.sort_values('counts', ascending=False).head(5)

trend_df = data_df["YearFrom"].value_counts().rename_axis('unique_values').reset_index(name='counts')

counts=[]
years = [*range(1500, 1803+1)]          

for year in years:
    count = trend_df.loc[trend_df['unique_values'] == year,'counts']
    if len(count) ==0:
        counts.append(0)
    else:
        counts.append(count.iloc[0])     

age_df = data_df.loc[data_df['VerdachteLeeftijd/AccusedAge'].notnull(), ['VerdachteLeeftijd/AccusedAge']]
age_df.index = pd.RangeIndex(len(age_df.index))

gender_df = data_df["VerdachteGeslacht/AccusedGandder"].value_counts().rename_axis('unique_values').reset_index(name='counts')

profession_df = data_df[profession_list].apply(pd.Series.value_counts).iloc[0].rename_axis('unique_values').reset_index(name='counts')
profession_df = profession_df.sort_values('counts', ascending=False).head(5)

place_df = data_df[place_list].apply(pd.Series.value_counts).iloc[0].rename_axis('unique_values').reset_index(name='counts')
place_df = place_df.sort_values('counts', ascending=False).head(5)

clicked_year_range = []
clicked_age_range = []
clicked_gender = None
clicked_profession = None
clicked_place = None
clicked_crime = None
clicked_punishment = None

In [9]:
trend_x_sc = bqp.LinearScale()
trend_y_sc = bqp.LinearScale()

trend_x_ax = bqp.Axis(
    label='Year', 
    scale=trend_x_sc, 
    grid_lines='solid', 
)

trend_y_ax = bqp.Axis(
    label='Number of Crimes', 
    scale=trend_y_sc, 
    orientation='vertical', 
)

trend_tt = bqp.Tooltip(fields=['x', 'y'])

trend = bqp.Scatter(x=years,
                    y=counts,
                    interactions={'click': 'select', 'hover':'tooltip'},
                    tooltip=trend_tt,
                    scales={'x': trend_x_sc, 'y': trend_y_sc} , 
                    colors=['blue'],
                    default_size=16) 

trend_line = bqp.Lines(x=years, y=counts, scales={'x': bqp.DateScale(), 'y': bqp.LinearScale()})

trend_fig = bqp.Figure(
    layout=Layout(height='240px', width='960px'),
    axes=[trend_x_ax, trend_y_ax], 
    marks=[trend, trend_line], 
    animation_duration=500)

age_x_sc = bqp.LinearScale()
age_y_sc = bqp.LinearScale()

age_tt = bqp.Tooltip(fields=['bin_start', 'bin_end', 'count'])

age_hist = bqp.Hist(
    sample = age_df['VerdachteLeeftijd/AccusedAge'],
    interactions={'click': 'select', 'hover':'tooltip'},
    tooltip=age_tt,
    scales={'sample': age_x_sc, 'count': age_y_sc}, 
)

age_x_ax = bqp.Axis(
    scale=age_x_sc
)

age_y_ax = bqp.Axis(
    scale=age_y_sc,
    orientation='vertical'
)

age_fig = bqp.Figure(
    title = 'Age',
    layout=Layout(height='240px',width='500px'),
    axes=[age_x_ax, age_y_ax],
    marks=[age_hist],
    animation_duration=500
)

gender_x_ord = bqp.OrdinalScale()
gender_y_sc = bqp.LinearScale()

gender_tt = bqp.Tooltip(fields=['x', 'y'])

gender_bar = bqp.Bars(
    x = gender_df['unique_values'],
    y = gender_df['counts'],
    interactions={'click': 'select', 'hover':'tooltip'},
    tooltip=gender_tt,
    scales={'x': gender_x_ord, 'y': gender_y_sc}, 
    orientation='horizontal'
)

gender_x_ax = bqp.Axis(
    scale=gender_y_sc
)

gender_y_ax = bqp.Axis(
    scale=gender_x_ord,
    orientation='vertical'
)

gender_fig = bqp.Figure(
    title = 'Gender',
    layout=Layout(height='180px',width='450px'),
    axes=[gender_x_ax, gender_y_ax],
    marks=[gender_bar],
    animation_duration=500
)


profession_x_ord = bqp.OrdinalScale()
profession_y_sc = bqp.LinearScale()

profession_x_ax = bqp.Axis(
    scale=profession_y_sc
)

profession_y_ax = bqp.Axis(
    scale=profession_x_ord,
    orientation='vertical'
)

profession_tt = bqp.Tooltip(fields=['x', 'y'])

profession_bar = bqp.Bars(
    x = profession_df['unique_values'],
    y = profession_df['counts'],
    interactions={'click': 'select', 'hover':'tooltip'},
    tooltip=profession_tt,
    scales={'x': profession_x_ord, 'y': profession_y_sc}, 
    orientation='horizontal'
)

profession_fig = bqp.Figure(
    title = 'Top 5 Profession',
    layout=Layout(height='180px',width='450px'),
    axes=[profession_x_ax, profession_y_ax],
    marks=[profession_bar],
    animation_duration=500,
    fig_margin={'top':60, 'bottom':60, 'left':100, 'right':60}
)

place_x_ord = bqp.OrdinalScale()
place_y_sc = bqp.LinearScale()

place_tt = bqp.Tooltip(fields=['x', 'y'])

place = bqp.Bars(
    x = place_df['unique_values'],
    y = place_df['counts'],
    interactions={'click': 'select', 'hover':'tooltip'},
    tooltip=place_tt,
    scales={'x': place_x_ord, 'y': place_y_sc}, 
    orientation='horizontal'
)

place_x_ax = bqp.Axis(
    scale=place_y_sc
)

place_y_ax = bqp.Axis(
    scale=place_x_ord,
    orientation='vertical'
)

place_fig = bqp.Figure(
    title = 'Top '+str(len(place_df))+' Place',
    layout=Layout(height='360px',width='500px'),
    axes=[place_x_ax, place_y_ax],
    marks=[place],
    animation_duration=500,
    fig_margin={'top':60, 'bottom':60, 'left':100, 'right':60}
)


crime_x_ord = bqp.OrdinalScale()
crime_y_sc = bqp.LinearScale()

crime_tt = bqp.Tooltip(fields=['x', 'y'])

crime_bar = bqp.Bars(
    x = crime_df['unique_values'],
    y = crime_df['counts'],
    interactions={'click': 'select', 'hover':'tooltip'},
    tooltip=crime_tt,
    scales={'x': crime_x_ord, 'y': crime_y_sc}, 
    orientation='horizontal'
)

crime_x_ax = bqp.Axis(
    scale=crime_y_sc
)

crime_y_ax = bqp.Axis(
    scale=crime_x_ord,
    orientation='vertical'
)

crime_fig = bqp.Figure(
    title = 'Top '+str(len(crime_df))+' Crime',
    layout=Layout(height='180px',width='550px'),
    axes=[crime_x_ax, crime_y_ax],
    marks=[crime_bar],
    animation_duration=500,
    fig_margin={'top':60, 'bottom':60, 'left':150, 'right':60}
)

punishment_x_ord = bqp.OrdinalScale()
punishment_y_sc = bqp.LinearScale()

punishment_x_ax = bqp.Axis(
    scale=punishment_y_sc
)

punishment_y_ax = bqp.Axis(
    scale=punishment_x_ord,
    orientation='vertical'
)

punishment_tt = bqp.Tooltip(fields=['x', 'y'])

punishment_bar = bqp.Bars(
    x = punishment_df['unique_values'],
    y = punishment_df['counts'],
    interactions={'click': 'select', 'hover':'tooltip'},
    tooltip=punishment_tt,
    scales={'x': punishment_x_ord, 'y': punishment_y_sc}, 
    orientation='horizontal'
)

punishment_fig = bqp.Figure(
    title = 'Top '+str(len(punishment_df))+' Punishment',
    layout=Layout(height='180px',width='550px'),
    axes=[punishment_x_ax, punishment_y_ax],
    marks=[punishment_bar],
    animation_duration=500,
    fig_margin={'top':60, 'bottom':60, 'left':150, 'right':60}
)

In [10]:
def filter_df_and_update_plots(age_flag=True, gender_flag=True, profession_flag=True, place_flag=True, crime_flag=True, punishment_flag=True):
    global years,age_df, gender_df, place_df, profession_df, crime_df, punishment_df 
    filtered_df = data_df
    if clicked_year_range != []:
        print(clicked_year_range)
        if clicked_year_range[0]==clicked_year_range[1]:
            filtered_df = filtered_df.loc[filtered_df["YearFrom"]==clicked_year_range[0]]
        else:
            filtered_df = filtered_df.loc[filtered_df["YearFrom"].isin(range(clicked_year_range[0], clicked_year_range[1]+1))]
    if clicked_age_range != []:
        print(clicked_age_range)
        if clicked_age_range[0]==clicked_age_range[1]:
            filtered_df = filtered_df.loc[filtered_df["VerdachteLeeftijd/AccusedAge"]==clicked_age_range[0]]
        else:
            filtered_df = filtered_df.loc[filtered_df["VerdachteLeeftijd/AccusedAge"].isin(range(int(clicked_age_range[0]), int(clicked_age_range[1]), 1))]
    if not(clicked_gender is None):
        print(clicked_gender)
        filtered_df = filtered_df.loc[filtered_df["VerdachteGeslacht/AccusedGandder"]==clicked_gender]
    if not(clicked_profession is None):
        print(clicked_profession)
        filtered_df = filtered_df.loc[filtered_df[clicked_profession]==1]
    if not(clicked_place is None):
        print(clicked_place)
        filtered_df = filtered_df.loc[filtered_df[clicked_place]==1]
    if not(clicked_crime is None):
        print(clicked_crime)
        filtered_df = filtered_df.loc[filtered_df[clicked_crime]==1]
    if not(clicked_punishment is None):
        print(clicked_punishment)
        filtered_df = filtered_df.loc[filtered_df[clicked_punishment]==1]
    
    trend_df = filtered_df["YearFrom"].value_counts().rename_axis('unique_values').reset_index(name='counts')
    
    counts = []
    if clicked_year_range == []:
        years = [*range(1500, 1803+1)]          
    else:
        years= [*range(clicked_year_range[0], clicked_year_range[1]+1)]
        
    for year in years:
        count = trend_df.loc[trend_df['unique_values'] == year,'counts']
        if len(count) ==0:
            counts.append(0)
        else:
            counts.append(count.iloc[0])     
            
    trend.x=years
    trend.y=counts
    trend_line.x=years
    trend_line.y=counts
    
    
#     if age_flag:
#         print('Age')
    age_df = filtered_df.loc[filtered_df['VerdachteLeeftijd/AccusedAge'].notnull(), ['VerdachteLeeftijd/AccusedAge']]
    age_df.index = pd.RangeIndex(len(age_df.index))
    age_hist.sample=age_df['VerdachteLeeftijd/AccusedAge']
    
#     if gender_flag:
#         print('Gender')
    gender_df = filtered_df["VerdachteGeslacht/AccusedGandder"].value_counts().rename_axis('unique_values').reset_index(name='counts')
    if len(gender_df)==0:
        gender_bar.x = []
        gender_bar.y = []
    else:
        gender_bar.x = gender_df['unique_values']
        gender_bar.y = gender_df['counts']
   
#     if profession_flag:
#         print('Profession')
    profession_df =filtered_df[profession_list].apply(pd.Series.value_counts)
    if len(profession_df)==0:
        profession_bar.x = []
        profession_bar.y = []
    else:
        profession_df = profession_df.iloc[0].rename_axis('unique_values').reset_index(name='counts')
        profession_df = profession_df.sort_values('counts', ascending=False).head(5)
        profession_bar.x = profession_df['unique_values']
        profession_bar.y = profession_df['counts']
    
#     if place_flag:
#         print('Place')
    place_df =filtered_df[place_list].apply(pd.Series.value_counts)
    if len(place_df)==0:
        place.x = []
        place.y = []
    else:
        place_df = place_df.iloc[0].rename_axis('unique_values').reset_index(name='counts')
        place_df = place_df.sort_values('counts', ascending=False).head(5)
        place.x = place_df['unique_values']
        place.y = place_df['counts']
    
#     if crime_flag:
#         print('Crime')
    crime_df =filtered_df[crime_list].apply(pd.Series.value_counts)
    if len(crime_df)==0:
        crime_bar.x = []
        crime_bar.y = []
    else:
        crime_df = crime_df.iloc[0].rename_axis('unique_values').reset_index(name='counts')
        crime_df = crime_df.sort_values('counts', ascending=False).head(5)
        crime_bar.x = crime_df['unique_values']
        crime_bar.y = crime_df['counts']
    
#     if punishment_flag:
#         print('Pubishment')
    punishment_df =filtered_df[punishment_list].apply(pd.Series.value_counts)
    if len(punishment_df)==0:
        punishment_bar.x = []
        punishment_bar.y = []
    else:
        punishment_df = punishment_df.iloc[0].rename_axis('unique_values').reset_index(name='counts')
        punishment_df = punishment_df.sort_values('counts', ascending=False).head(5)
        punishment_bar.x = punishment_df['unique_values']
        punishment_bar.y = punishment_df['counts']
    print('------')
    
def refresh_filters(flag=True, age_flag=True, gender_flag=True, profession_flag=True, place_flag=True,crime_flag=True, punishment_flag=True):
    filter_list = [years_filter, age_filter, gender_filter, profession_filter,place_filter, crime_filter, punishment_filter]
    filter_box.children =(widgets.Label(value="Selected filters, click to unselect"),)+tuple([f for f in filter_list if f.description != ""])
#     if flag:
    filter_df_and_update_plots(age_flag, gender_flag, profession_flag, place_flag,crime_flag, punishment_flag)

def click_trend(clicked):
    global clicked_year_range
    if clicked['new'] is None or len(clicked['new']) == 0:
        pass
    else:    
        clicked_year_range = [years[clicked['new'][0]], years[clicked['new'][0]]]
        trend_x_sc.min = clicked_year_range[0]
        trend_x_sc.max = clicked_year_range[1]
        year_range.value=(clicked_year_range[0], clicked_year_range[1])
    
def select_years(y):
    global clicked_year_range
    if year_range.value[0] == year_range.min and year_range.value[1] == year_range.max:
        clicked_year_range = []
        years_filter.description=''
        trend_x_sc.min = year_range.min
        trend_x_sc.max = year_range.max
        refresh_filters(flag=False)
    else:
        clicked_year_range = [year_range.value[0], year_range.value[1]]
        years_filter.description=str(clicked_year_range[0])+'~'+str(clicked_year_range[1])
        trend_x_sc.min = clicked_year_range[0]
        trend_x_sc.max = clicked_year_range[1]
        refresh_filters()

def click_age_hist(clicked):
    global clicked_age_range
    if clicked['new'] is None or len(clicked['new']) == 0:
        pass
    else:
        age_series = age_df[age_df.index.isin(clicked['new'])]['VerdachteLeeftijd/AccusedAge']
        clicked_age_range = [age_series.min(), age_series.max()]
        age_filter.description='Age'+str(clicked_age_range[0])+'~'+str(clicked_age_range[1])
        refresh_filters(age_flag=False)
    
def click_gender_bar(clicked):
    global clicked_gender 
    if clicked['new'] is None or len(clicked['new']) == 0:
        pass
    else:
        clicked_gender = gender_df.iloc[clicked['new'][0]]['unique_values']
        gender_filter.description='Gender: '+clicked_gender
        refresh_filters(age_flag=False,gender_flag=False)
    
def click_profession_bar(clicked):
    global clicked_profession
    if clicked['new'] is None or len(clicked['new']) == 0:
        pass
    else:
        clicked_profession = profession_df.iloc[clicked['new'][0]]['unique_values']
        profession_filter.description='Profession: '+clicked_profession
        refresh_filters(age_flag=False,gender_flag=False,profession_flag=False)

def click_place(clicked):
    global clicked_place
    if clicked['new'] is None or len(clicked['new']) == 0:
        pass
    else:
        clicked_place = place_df.iloc[clicked['new'][0]]['unique_values']
        place_filter.description='Place: '+clicked_place
        refresh_filters(age_flag=False,gender_flag=False,profession_flag=False, place_flag=False)    
    
def click_crime_bar(clicked):
    global clicked_crime
    if clicked['new'] is None or len(clicked['new']) == 0:
        pass
    else:
        clicked_crime = crime_df.iloc[clicked['new'][0]]['unique_values']
        crime_filter.description='Crime: '+clicked_crime
        refresh_filters(age_flag=False,gender_flag=False,profession_flag=False,place_flag=False,crime_flag=False)
    
def click_punishment_bar(clicked):
    global clicked_punishment
    if clicked['new'] is None or len(clicked['new']) == 0:
        pass
    else:
        clicked_punishment = punishment_df.iloc[clicked['new'][0]]['unique_values']
        punishment_filter.description= 'Punishment: '+clicked_punishment
        refresh_filters(age_flag=False,gender_flag=False,profession_flag=False,place_flag=False,crime_flag=False,punishment_flag=False)

def click_years_filter(clicked):
    global clicked_year_range
    clicked_year_range = []
    years_filter.description=''
    year_range.value=[1500, 1803]
    trend_x_sc.min = 1500
    trend_x_sc.max = 1803
    refresh_filters()
    
def click_age_filter(clicked):
    global clicked_age_range 
    clicked_age_range = []
    age_filter.description=''
    refresh_filters(age_flag=False)
    
def click_gender_filter(clicked):
    global clicked_gender
    clicked_gender = None
    gender_filter.description=''
    refresh_filters(age_flag=False,gender_flag=False)
    
def click_profession_filter(clicked):
    global clicked_profession
    clicked_profession = None
    profession_filter.description=''
    refresh_filters(age_flag=False,gender_flag=False,profession_flag=False)

def click_place_filter(clicked):
    global clicked_place
    clicked_place = None
    place_filter.description=''
    refresh_filters(age_flag=False,gender_flag=False,profession_flag=False,place_flag=False)
    
def click_crime_filter(clicked):
    global clicked_crime
    clicked_crime = None
    crime_filter.description=''
    refresh_filters(age_flag=False,gender_flag=False,profession_flag=False,place_flag=False,crime_flag=False)
    
def click_punishment_filter(clicked):
    global clicked_punishment
    clicked_punishment = None
    punishment_filter.description=''
    refresh_filters(age_flag=False,gender_flag=False,profession_flag=False,place_flag=False,crime_flag=False,punishment_flag=False)

In [11]:
#assign widget actions to functions
trend.observe(click_trend, 'selected')
year_range.observe(select_years, names = 'value')
age_hist.observe(click_age_hist, 'selected')
gender_bar.observe(click_gender_bar, 'selected')
profession_bar.observe(click_profession_bar, 'selected')
place.observe(click_place, 'selected')
crime_bar.observe(click_crime_bar, 'selected')
punishment_bar.observe(click_punishment_bar, 'selected')

years_filter.on_click(click_years_filter)
age_filter.on_click(click_age_filter)
gender_filter.on_click(click_gender_filter)
profession_filter.on_click(click_profession_filter)
place_filter.on_click(click_place_filter)
crime_filter.on_click(click_crime_filter)
punishment_filter.on_click(click_punishment_filter)

In [12]:
#display all
filter_box = HBox(
    children=(widgets.Label(value="Selected filters, click to unselect"),),
)

top_right_box = VBox(
    children=(trend_fig, year_range),
)

top_box = HBox(
    children=(age_fig,top_right_box),
)

bottom_left_box = VBox(
    children=(gender_fig, profession_fig),
)


bottom_right_box = VBox(
    children=(crime_fig, punishment_fig),
)

bottom_box = HBox(
    children=(bottom_left_box, m,bottom_right_box),
)

vert_box  = VBox(
    children=(filter_box, top_box, bottom_box),
)

display (vert_box)

[1597, 1803]
------
[1692, 1803]
------
------
man
------
[1722, 1803]
man
------
[1722, 1725]
man
------
[1620, 1725]
man
------
[1620, 1725]
------
------
------
s-Hertogenbosch
------
------
Servant
------
------
[1553, 1803]
------
[1502, 1803]
------
[1588, 1803]
------
[1612, 1803]
------
[1544, 1803]
------
[1501, 1803]
------
------
Soldier
------
------
